# 1. Demo cosine similarity

## Sử dụng keyBERT để trích xuất keywords

In [250]:
from keybert import KeyBERT

kw_model = KeyBERT(model="paraphrase-multilingual-MiniLM-L12-v2")

## Preprocess Text Function

In [251]:
#Lowercase
def lowercase(text):
    return text.lower()

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

from underthesea import word_tokenize

def tokenize(text):
    return word_tokenize(text, format="text")

import urllib
# Download vietnamese stop words
data=urllib.request.urlopen('https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt').read()
#data = open("vietnamese-stopwords-dash.txt", "r", encoding="utf8").read()

# Read stop words
stopwords= [ x for x in data.splitlines() ]
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stopwords])

import re
def remove_html_url(text):
    return re.sub(r'http\S+', '', text)

import emoji

def remove_emoji(text):
    def get_emoji_regexp():
        # Sort emoji by length to make sure multi-character emojis are
    # matched first
        emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
        pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
        return re.compile(pattern)

    exp = get_emoji_regexp()
    return exp.sub(u'', text)

def preprocess_text(text):
    # '''Input : String, output : String'''
      text_url = remove_html_url(text)
      text_emoji = remove_emoji(text_url)
    
      text_lowercase = lowercase(text_emoji)
      text_punctuation = remove_punctuation(text_lowercase)
      
      text_tokenize = tokenize(text_punctuation)
      text_stopwords = remove_stopwords(text_tokenize)
  
      return text_stopwords

## Extract keywords from 3 documents

In [252]:
doc_1 = """
      Đặc điểm nổi bật của iPhone 15 Pro Max
      Tăng độ cứng cáp và tối ưu khối lượng với chất liệu Titan
      Bứt phá mọi giới hạn về hiệu năng nhờ chip A17 Pro
      Phiên bản duy nhất cải tiến camera tele 5x
      Tích hợp camera 48 MP
      Nút tác vụ (Action Button) thay thế cần gạt rung
      Chuyển đổi cổng sạc USB-C, truyền tải dữ liệu tốc độ cao
      """
doc_1_pre = preprocess_text(doc_1)
keywords_1 = kw_model.extract_keywords(doc_1_pre, top_n=10, use_mmr=True)
keywords_1 = [word[0] for word in keywords_1]
keywords_1

['iphone',
 'chip',
 'tốc_độ',
 'cứng_cáp',
 'titan',
 'camera',
 'a17',
 'max',
 'action_button',
 'cổng']

In [253]:
doc_2 = """
      Đặc điểm nổi bật của iPhone 15 128GB
      Sử dụng kiểu hình notch Dynamic Island 
      Ra mắt với 5 phiên bản màu sắc và được làm từ kính pha màu
      Sử dụng Chip A16 Bionic cho hiệu năng vượt trội
      Trang bị camera chính 48 MP, Telephoto 2x cho khả năng chụp ảnh sắc nét
      Màn hình OLED Super Retina XDR cho chất lượng đồ họa lý tưởng
      Hỗ trợ sẵn hệ điều hành iOS 17 mới nhất 2023
      """
doc_2_pre = preprocess_text(doc_2)
keywords_2 = kw_model.extract_keywords(doc_2_pre, top_n=10, use_mmr=True)
keywords_2 = [word[0] for word in keywords_2]
keywords_2

['camera',
 'iphone',
 'retina',
 'a16',
 'chip',
 'gb',
 'màu',
 'island',
 'bionic',
 'dynamic']

In [254]:
doc_3 = """
      Tạo hình vuông vức cùng hiệu ứng mặt lưng cuốn hút
      Màn hình lớn đi kèm tấm nền AMOLED
      Camera 64 MP đáp ứng tốt nhu cầu chụp ảnh
      Cấu hình mạnh mẽ trong tầm giá
      Pin 5000 mAh và có đi kèm sạc nhanh 67 W
      """
doc_3_pre = preprocess_text(doc_3)
keywords_3 = kw_model.extract_keywords(doc_3_pre, top_n=10, use_mmr=True)
keywords_3 = [word[0] for word in keywords_3]
keywords_3

['camera',
 'nhanh',
 '5000',
 'vuông_vức',
 '64',
 'giá',
 'amoled',
 'pin',
 'mặt',
 'lưng']

## Cosine similarity matrix

In [255]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")

Ta nhầm là sử dụng tfidf, sử dụng count vectorizer để trích xuất features từ list keywords

In [256]:
count = CountVectorizer()

In [257]:
count_matrix = count.fit_transform([" ".join(keywords_1), " ".join(keywords_2), " ".join(keywords_3)])

In [258]:
csm_count = cosine_similarity(count_matrix, count_matrix)

In [259]:
csm_count

array([[1. , 0.3, 0.1],
       [0.3, 1. , 0.1],
       [0.1, 0.1, 1. ]])

# 2. Test trên tập dataset điện thoại

## Load data

In [264]:
import pandas as pd
data = pd.read_csv("tgdd_product_description_grouped_preprocessed_labeled.csv")
data.sample(5)

,Unnamed: 0,name,description,label,x_corpus,y_label
131,131,/may-tinh-bang/xiaomi-redmi-pad-se,xiaomi redmi pad se sở_hữu thiết_kế đặc_trưng ...,"['bảng_xiaomi', 'pad_se', 'pad', 'máy_tính', '...","['xiaomi', 'redmi', 'pad', 'se', 'sở_hữu', 'th...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
63,63,/dtdd/samsung-galaxy-a14-5g,galaxy a14 cảm_hứng thiết_kế mẫu điện_thoại sa...,"['samsung', 'galaxy', 'android', 'a14', 'hd', ...","['galaxy', 'a14', 'cảm_hứng', 'thiết_kế', 'mẫu...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ..."
73,73,/dtdd/samsung-galaxy-s23-ultra,thiết_kế samsung galaxy s23 ultra thừa_hưởng k...,"['samsung_tích_hợp', 'samsung', 'galaxy', 'and...","['thiết_kế', 'samsung', 'galaxy', 's23', 'ultr...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, ..."
55,55,/dtdd/realme-c53,ngoại_hình máy vô_cùng ấn_tượng máy thừa_hưởng...,"['máy', 'camera', 'máy_lạnh', 'điện_thoại', 'c...","['ngoại_hình', 'máy', 'vô_cùng', 'ấn_tượng', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
61,61,/dtdd/samsung-galaxy-a05s-6gb,galaxy a05s 6 gb cảm_hứng thiết_kế phiên_bản f...,"['galaxy', 'galaxy_a', 'galaxy_s23', 'samsung'...","['galaxy', 'a05s', '6', 'gb', 'cảm_hứng', 'thi...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Chuyển label keywords về list objects

In [265]:
# Import the ast module
import ast
data['label'] = data['label'].apply(lambda x: ast.literal_eval(x))
data['label'] = data['label'].apply(lambda x: " ".join(x))
data['label']

0      camera_kép camera iphone iphone_xr smartphone ...
1      smartphone_apple apple iphone ipad ip68 a14 a1...
2      apple iphone chip ip68 a15 sensor nhôm gb pixe...
3      iphone apple điện_thoại điện_ảnh máy app chip ...
4      iphone apple_tính apple smartphone nhôm máy kh...
                             ...                        
129    wifi_tích wifi tcl nhôm_nguyên internet lcd má...
130    cao_xiaomi gb sẵn_sàng graphite xiaomi cao_cấp...
131    bảng_xiaomi pad_se pad máy_tính redmi mượt_mà ...
132    nhanh_chóng bảng_xiaomi độ_chính_xác máy_tính ...
133    gần_gũi bảng_xiaomi redmi nhu_cầu công_suất ph...
Name: label, Length: 134, dtype: object

## Cosine similarity matrix

Ta nhầm là sử dụng tfidf, sử dụng count vectorizer để trích xuất features từ list keywords

In [266]:
count = CountVectorizer()
count_matrix = count.fit_transform(data['label'])
csm_count = cosine_similarity(count_matrix, count_matrix)

## Test Recommendations

Tạo indices với key là name (tên của điện thoại)

In [267]:
# recommmendation function
indices = pd.Series(data.index, index=data.name)

In [268]:
indices

name
/dtdd/iphone-11                             0
/dtdd/iphone-12                             1
/dtdd/iphone-13                             2
/dtdd/iphone-14                             3
/dtdd/iphone-14-plus                        4
                                         ... 
/may-tinh-bang/tcl-tab-11                 129
/may-tinh-bang/xiaomi-pad-6               130
/may-tinh-bang/xiaomi-redmi-pad-se        131
/may-tinh-bang/xiaomi-redmi-pad-se-4gb    132
/may-tinh-bang/xiaomi-redmi-pad-se-8gb    133
Length: 134, dtype: int64

Hàm recommend

In [269]:
def recommend(post, csm=csm_count):        # choosing this csm as it covers both aspects
    idx = indices[post]
    score_series = list(enumerate(csm[idx]))
    score_series = sorted(score_series, key=lambda x: x[1], reverse=True)
    score_series = score_series[1:11]   # not recommending the original post itself, starting from 1
    post_indices = [i[0] for i in score_series]
    return data.loc[post_indices]

In [270]:
data_rec = recommend('/dtdd/samsung-galaxy-a04s')
data_rec[['name', 'label']]

,name,label
53,/dtdd/realme-c30s-3gb-64gb,máy khung màn_hình kết_hợp tích_hợp thiết_kế k...
85,/dtdd/vivo-y22s-8gb?utm_flashsale=1,màu_sắc vivo_y22s nhẹ ánh_sáng xanh_dương y22s...
30,/dtdd/oppo-a17k,điện_thoại máy cứng_cáp nhựa chip nhanh_chóng ...
86,/dtdd/vivo-y35-2022?utm_flashsale=1,điện_thoại máy 2022 cứng_cáp năng_lượng đa_nhi...
36,/dtdd/oppo-a79-5g,thủy_tinh_hữu_cơ hình_chữ_nhật giao_tranh sở_h...
49,/dtdd/realme-10,máy điện_thoại độ_chính_xác thủy_tinh_hữu_cơ t...
47,/dtdd/oppo-reno8-t-5g?utm_flashsale=1,cao_cấp thiết_kế tổng_kết thủy_tinh_hữu_cơ đa_...
94,/dtdd/xiaomi-redmi-a3,camera_hình camera redmi_a3 redmi thoải_mái ch...
29,/dtdd/oppo-a17?utm_flashsale=1,nhựa mềm máy nhẹ chất_lượng nhanh_chóng dễ_dàn...
63,/dtdd/samsung-galaxy-a14-5g,samsung galaxy android a14 hd điện_thoại máy c...


Mô tả quá trình recommend sử dụng matrix

In [271]:
post = '/dtdd/iphone-14-plus'

In [272]:
idx = indices[post]
idx

4

In [273]:
score_series = list(enumerate(csm_count[idx]))
score_series

[(0, 0.2997202997204493),
 (1, 0.2659159586619983),
 (2, 0.3360000000000002),
 (3, 0.4360000000000003),
 (4, 1.0000000000000007),
 (5, 0.32853916736992567),
 (6, 0.3720000000000002),
 (7, 0.2160000000000001),
 (8, 0.2600000000000001),
 (9, 0.24000000000000013),
 (10, 0.19986296675335338),
 (11, 0.2168432129926576),
 (12, 0.29804059091439034),
 (13, 0.19688854885731927),
 (14, 0.21936339884283254),
 (15, 0.16484511834894683),
 (16, 0.2231436834155872),
 (17, 0.2241893609545677),
 (18, 0.2826489181987154),
 (19, 0.2472393025537662),
 (20, 0.24678382369818658),
 (21, 0.24693395463717804),
 (22, 0.28509866713122334),
 (23, 0.23306957882020657),
 (24, 0.17483467670660893),
 (25, 0.21915421836818777),
 (26, 0.21634097809573394),
 (27, 0.16960529899389498),
 (28, 0.21721803209997959),
 (29, 0.3400000000000002),
 (30, 0.3400000000000002),
 (31, 0.30917219141796787),
 (32, 0.2949645430092452),
 (33, 0.2271847336988261),
 (34, 0.28527988955517947),
 (35, 0.29655979568764945),
 (36, 0.30181636371

In [274]:
score_series = sorted(score_series, key=lambda x: x[1], reverse=True)
score_series

[(4, 1.0000000000000007),
 (3, 0.4360000000000003),
 (6, 0.3720000000000002),
 (43, 0.3560000000000002),
 (80, 0.3520000000000002),
 (86, 0.3520000000000002),
 (103, 0.3520000000000002),
 (42, 0.34570535882735626),
 (65, 0.34415494032603927),
 (47, 0.3414013961523854),
 (63, 0.34087913042643686),
 (29, 0.3400000000000002),
 (30, 0.3400000000000002),
 (37, 0.3395352861284307),
 (2, 0.3360000000000002),
 (49, 0.3360000000000002),
 (96, 0.3360000000000002),
 (51, 0.3320000000000002),
 (98, 0.3293470564132538),
 (53, 0.3290778592414298),
 (5, 0.32853916736992567),
 (74, 0.3280000000000002),
 (85, 0.3280000000000002),
 (41, 0.3240000000000002),
 (69, 0.3240000000000002),
 (88, 0.3225161494664522),
 (52, 0.3200000000000002),
 (56, 0.3200000000000002),
 (71, 0.3200000000000002),
 (68, 0.31957418477478455),
 (57, 0.3195048252113469),
 (81, 0.3177723290534173),
 (72, 0.31600000000000017),
 (75, 0.31600000000000017),
 (50, 0.31200000000000017),
 (84, 0.31200000000000017),
 (106, 0.31200000000000

In [275]:
score_series = score_series[1:11]   # not recommending the original post itself, starting from 1
score_series

[(3, 0.4360000000000003),
 (6, 0.3720000000000002),
 (43, 0.3560000000000002),
 (80, 0.3520000000000002),
 (86, 0.3520000000000002),
 (103, 0.3520000000000002),
 (42, 0.34570535882735626),
 (65, 0.34415494032603927),
 (47, 0.3414013961523854),
 (63, 0.34087913042643686)]

In [276]:
post_indices = [i[0] for i in score_series]
post_indices

[3, 6, 43, 80, 86, 103, 42, 65, 47, 63]

In [277]:
data.loc[post_indices][['name', 'label']]

,name,label
3,/dtdd/iphone-14,iphone apple điện_thoại điện_ảnh máy app chip ...
6,/dtdd/iphone-14-pro-max,iphone smartphone apple gb pixels max zoom_qua...
43,/dtdd/oppo-reno10-pro-plus,camera_chính camera_góc camera chụp hình_ảnh ả...
80,/dtdd/vivo-v29-5g,camera tốc_độ pixels chụp vivo_v27e nhanh_chón...
86,/dtdd/vivo-y35-2022?utm_flashsale=1,điện_thoại máy 2022 cứng_cáp năng_lượng đa_nhi...
103,/may-tinh-bang/ipad-gen-10,ipad ipados nhà_apple máy_tính cài_đặt tablet ...
42,/dtdd/oppo-reno10-pro,camera máy hd_kích_thước portrait_cam tốc_độ c...
65,/dtdd/samsung-galaxy-a15-5g,samsung galaxy galaxy_a điện_thoại camera_góc ...
47,/dtdd/oppo-reno8-t-5g?utm_flashsale=1,cao_cấp thiết_kế tổng_kết thủy_tinh_hữu_cơ đa_...
63,/dtdd/samsung-galaxy-a14-5g,samsung galaxy android a14 hd điện_thoại máy c...


# 3. Test add user request

Extract keywords from user request

In [278]:
from keybert import KeyBERT

kw_model = KeyBERT(model="paraphrase-multilingual-MiniLM-L12-v2")

In [279]:
doc = """
      Tôi muốn một chiếc điện thoại giá rẻ, đủ để chơi game, nghe nhạc chụp hình. Hệ điều hành phải là android. Giá từ 3 - 5 triệu
      """
doc_pre = preprocess_text(doc)
keywords = kw_model.extract_keywords(doc_pre, top_n=10, use_mmr=True)
keywords = [word[0] for word in keywords]
keywords

['android',
 'rẻ',
 'giá',
 'điện_thoại',
 'game',
 'triệu',
 'chụp',
 'điều_hành',
 'đủ',
 'một']

Recommend function with keywords

In [280]:
def recommend(keywords):
    import pandas as pd
    data = pd.read_csv("tgdd_product_description_grouped_preprocessed_labeled.csv")
    
    # Import the ast module
    import ast
    data['label'] = data['label'].apply(lambda x: ast.literal_eval(x))
    data['label'] = data['label'].apply(lambda x: " ".join(x))
    
    user_request = {"name": "user_request", "label": " ".join(keywords)}
    data = pd.concat([data, pd.DataFrame([user_request])], ignore_index=True)

    count = CountVectorizer()
    count_matrix = count.fit_transform(data['label'])
    csm_count = cosine_similarity(count_matrix, count_matrix)
    
    indices = pd.Series(data.index, index=data.name)
    idx = indices["user_request"]
    score_series = list(enumerate(csm_count[idx]))
    score_series = sorted(score_series, key=lambda x: x[1], reverse=True)
    score_series = score_series[1:11]   # not recommending the original post itself, starting from 1
    post_indices = [i[0] for i in score_series]
    return data.loc[post_indices]

In [281]:
recommend(keywords)

,Unnamed: 0,name,description,label,x_corpus,y_label
82,82.0,/dtdd/vivo-y02t,điện_thoại vivo thiết_kế đơn_giản tinh_tế thân...,camera pixels video hình_ảnh flash chụp hd ảnh...,"['điện_thoại', 'vivo', 'thiết_kế', 'đơn_giản',...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ..."
12,12.0,/dtdd/itel-l6502,itel l6502 ra_mắt tiêu_dùng màu_sắc trẻ trung_...,điện_thoại nhu_cầu tiêu_dùng tinh_chỉnh máy ng...,"['itel', 'l6502', 'ra_mắt', 'tiêu_dùng', 'màu_...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
62,62.0,/dtdd/samsung-galaxy-a14,samsung galaxy a14 4 g thiết_kế thừa_hưởng vẻ ...,samsung galaxy smartphone android điện_thoại m...,"['samsung', 'galaxy', 'a14', '4', 'g', 'thiết_...","[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
33,33.0,/dtdd/oppo-a57-128gb?utm_flashsale=1,điện_thoại oppo_a màn_hình giọt mặt khung viền...,gb hd pixels f22_cảm hdr máy điện_thoại g35 10...,"['điện_thoại', 'oppo_a', 'màn_hình', 'giọt', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."
57,57.0,/dtdd/realme-c60,realme c60 bắt_gặp thiết_kế quen_thuộc khung v...,máy điện_thoại c60 gb t612 hd tốc_độ chip thiế...,"['realme', 'c60', 'bắt_gặp', 'thiết_kế', 'quen...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
129,129.0,/may-tinh-bang/tcl-tab-11,tcl tab 11 wifi máy_tính bảng thiết_kế vuông_v...,wifi_tích wifi tcl nhôm_nguyên internet lcd má...,"['tcl', 'tab', '11', 'wifi', 'máy_tính', 'bảng...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
63,63.0,/dtdd/samsung-galaxy-a14-5g,galaxy a14 cảm_hứng thiết_kế mẫu điện_thoại sa...,samsung galaxy android a14 hd điện_thoại máy c...,"['galaxy', 'a14', 'cảm_hứng', 'thiết_kế', 'mẫu...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ..."
83,83.0,/dtdd/vivo-y03-64gb,thiết_kế vuông vức vivo y03 64 gb sản_phẩm điệ...,máy cpu microsd điện_thoại nhanh_chóng gb nhựa...,"['thiết_kế', 'vuông', 'vức', 'vivo', 'y03', '6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
53,53.0,/dtdd/realme-c30s-3gb-64gb,realme c30s máy thiết_kế đẹp nổi_bật tầm_giá n...,máy khung màn_hình kết_hợp tích_hợp thiết_kế k...,"['realme', 'c30s', 'máy', 'thiết_kế', 'đẹp', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
60,60.0,/dtdd/samsung-galaxy-a05-4gb,samsung galaxy_a05 mục_tiêu đem sang_trọng hợp...,samsung galaxy_a05 galaxy android điện_thoại_d...,"['samsung', 'galaxy_a05', 'mục_tiêu', 'đem', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


Demo steps to recommend

In [282]:
import pandas as pd
data = pd.read_csv("tgdd_product_description_grouped_preprocessed_labeled.csv")

# Import the ast module
import ast
data['label'] = data['label'].apply(lambda x: ast.literal_eval(x))
data['label'] = data['label'].apply(lambda x: " ".join(x))
data['label']

0      camera_kép camera iphone iphone_xr smartphone ...
1      smartphone_apple apple iphone ipad ip68 a14 a1...
2      apple iphone chip ip68 a15 sensor nhôm gb pixe...
3      iphone apple điện_thoại điện_ảnh máy app chip ...
4      iphone apple_tính apple smartphone nhôm máy kh...
                             ...                        
129    wifi_tích wifi tcl nhôm_nguyên internet lcd má...
130    cao_xiaomi gb sẵn_sàng graphite xiaomi cao_cấp...
131    bảng_xiaomi pad_se pad máy_tính redmi mượt_mà ...
132    nhanh_chóng bảng_xiaomi độ_chính_xác máy_tính ...
133    gần_gũi bảng_xiaomi redmi nhu_cầu công_suất ph...
Name: label, Length: 134, dtype: object

In [283]:
user_request = {"name": "user_request", "label": " ".join(keywords)}
data = pd.concat([data, pd.DataFrame([user_request])], ignore_index=True)

In [284]:
data.tail()

,Unnamed: 0,name,description,label,x_corpus,y_label
130,130.0,/may-tinh-bang/xiaomi-pad-6,xiaomi pad 6 máy_tính bảng thiết_kế tinh_tế ca...,cao_xiaomi gb sẵn_sàng graphite xiaomi cao_cấp...,"['xiaomi', 'pad', '6', 'máy_tính', 'bảng', 'th...","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
131,131.0,/may-tinh-bang/xiaomi-redmi-pad-se,xiaomi redmi pad se sở_hữu thiết_kế đặc_trưng ...,bảng_xiaomi pad_se pad máy_tính redmi mượt_mà ...,"['xiaomi', 'redmi', 'pad', 'se', 'sở_hữu', 'th...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
132,132.0,/may-tinh-bang/xiaomi-redmi-pad-se-4gb,máy_tính bảng xiaomi redmi pad_se thiết_kế hiệ...,nhanh_chóng bảng_xiaomi độ_chính_xác máy_tính ...,"['máy_tính', 'bảng', 'xiaomi', 'redmi', 'pad_s...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
133,133.0,/may-tinh-bang/xiaomi-redmi-pad-se-8gb,redmi pad_se thiết_kế dựa cảm_hứng xu_hướng vu...,gần_gũi bảng_xiaomi redmi nhu_cầu công_suất ph...,"['redmi', 'pad_se', 'thiết_kế', 'dựa', 'cảm_hứ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
134,NaN,user_request,NaN,android rẻ giá điện_thoại game triệu chụp điều...,NaN,NaN


In [285]:
import numpy as np

count = CountVectorizer()
count_matrix = count.fit_transform(data['label'])
csm_count = cosine_similarity(count_matrix, count_matrix)

In [286]:
# recommmendation function
indices = pd.Series(data.index, index=data.name)

In [287]:
indices

name
/dtdd/iphone-11                             0
/dtdd/iphone-12                             1
/dtdd/iphone-13                             2
/dtdd/iphone-14                             3
/dtdd/iphone-14-plus                        4
                                         ... 
/may-tinh-bang/xiaomi-pad-6               130
/may-tinh-bang/xiaomi-redmi-pad-se        131
/may-tinh-bang/xiaomi-redmi-pad-se-4gb    132
/may-tinh-bang/xiaomi-redmi-pad-se-8gb    133
user_request                              134
Length: 135, dtype: int64

In [288]:
idx = indices['user_request']
score_series = list(enumerate(csm_count[idx]))
score_series = sorted(score_series, key=lambda x: x[1], reverse=True)
score_series = score_series[1:11]
post_indices = [i[0] for i in score_series]
data.loc[post_indices]

,Unnamed: 0,name,description,label,x_corpus,y_label
82,82.0,/dtdd/vivo-y02t,điện_thoại vivo thiết_kế đơn_giản tinh_tế thân...,camera pixels video hình_ảnh flash chụp hd ảnh...,"['điện_thoại', 'vivo', 'thiết_kế', 'đơn_giản',...","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, ..."
12,12.0,/dtdd/itel-l6502,itel l6502 ra_mắt tiêu_dùng màu_sắc trẻ trung_...,điện_thoại nhu_cầu tiêu_dùng tinh_chỉnh máy ng...,"['itel', 'l6502', 'ra_mắt', 'tiêu_dùng', 'màu_...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
62,62.0,/dtdd/samsung-galaxy-a14,samsung galaxy a14 4 g thiết_kế thừa_hưởng vẻ ...,samsung galaxy smartphone android điện_thoại m...,"['samsung', 'galaxy', 'a14', '4', 'g', 'thiết_...","[1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
33,33.0,/dtdd/oppo-a57-128gb?utm_flashsale=1,điện_thoại oppo_a màn_hình giọt mặt khung viền...,gb hd pixels f22_cảm hdr máy điện_thoại g35 10...,"['điện_thoại', 'oppo_a', 'màn_hình', 'giọt', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ..."
57,57.0,/dtdd/realme-c60,realme c60 bắt_gặp thiết_kế quen_thuộc khung v...,máy điện_thoại c60 gb t612 hd tốc_độ chip thiế...,"['realme', 'c60', 'bắt_gặp', 'thiết_kế', 'quen...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
129,129.0,/may-tinh-bang/tcl-tab-11,tcl tab 11 wifi máy_tính bảng thiết_kế vuông_v...,wifi_tích wifi tcl nhôm_nguyên internet lcd má...,"['tcl', 'tab', '11', 'wifi', 'máy_tính', 'bảng...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
63,63.0,/dtdd/samsung-galaxy-a14-5g,galaxy a14 cảm_hứng thiết_kế mẫu điện_thoại sa...,samsung galaxy android a14 hd điện_thoại máy c...,"['galaxy', 'a14', 'cảm_hứng', 'thiết_kế', 'mẫu...","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ..."
83,83.0,/dtdd/vivo-y03-64gb,thiết_kế vuông vức vivo y03 64 gb sản_phẩm điệ...,máy cpu microsd điện_thoại nhanh_chóng gb nhựa...,"['thiết_kế', 'vuông', 'vức', 'vivo', 'y03', '6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
53,53.0,/dtdd/realme-c30s-3gb-64gb,realme c30s máy thiết_kế đẹp nổi_bật tầm_giá n...,máy khung màn_hình kết_hợp tích_hợp thiết_kế k...,"['realme', 'c30s', 'máy', 'thiết_kế', 'đẹp', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
60,60.0,/dtdd/samsung-galaxy-a05-4gb,samsung galaxy_a05 mục_tiêu đem sang_trọng hợp...,samsung galaxy_a05 galaxy android điện_thoại_d...,"['samsung', 'galaxy_a05', 'mục_tiêu', 'đem', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
